<a href="https://colab.research.google.com/github/SidduVishnuPriya/CAPTCHA_Generator-using-HTML-CSS-JS/blob/main/virtual_insurance_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain-experimental langchain-community langchain-openai openai chromadb pypdf sentence_transformers gradio langchain-together

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of langchain-together to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-together to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of together to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is still looking at multiple versions of together to determine which ver

In [2]:
import os
from google.colab import userdata
os.environ["TOGETHER_API_KEY"] = userdata.get('TOGETHER_API_KEY') # user your together ai api key here
# go to https://docs.together.ai/docs/quickstart to register

#document loader
from langchain_community.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

# Doc splitting
from langchain.text_splitter import RecursiveCharacterTextSplitter

# vector store
from langchain_community.vectorstores import Chroma

#llm
from langchain_together import Together

SecretNotFoundError: Secret TOGETHER_API_KEY does not exist.

In [ ]:
loader = PyPDFDirectoryLoader("Insurance Exam Documents/")
docs = loader.load()

In [ ]:
len(docs)

In [ ]:
docs[16]

In [ ]:
def split_docs(documents, chunk_size=500, chunk_overlap=100):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

In [ ]:
pages = split_docs(docs)
len(pages)

In [ ]:
pages[500].page_content

In [ ]:
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

db = Chroma.from_documents(pages, embedding_function)

In [ ]:
query = "what is risk management?"
response = db.similarity_search(query)
print(response[0])

In [ ]:
response = db.similarity_search_with_relevance_scores(query)
response

# Build RAG using above

In [ ]:
llm = Together(
    model="meta-llama/Llama-2-70b-chat-hf",
    max_tokens=256,
    temperature=0.1,
    top_k=1
)

## similarity score helps to fetch only relevant results

In [ ]:
retriever = db.as_retriever(similarity_score_threshold = 0.6)

In [ ]:
from langchain.prompts import PromptTemplate
prompt_template = """
    You are a helpful insurance agent. Answer insurance related question using the given context.
    Context: {context}
    User Question: {question}
    Answer in less than 100 words
    If you don't have any context and are unsure of the answer, reply that you don't know about this topic.
    """

PROMPT = PromptTemplate(template = f"[INST] {prompt_template} [/INST]", input_variables=["context", "question"])

In [ ]:
from langchain.chains import RetrievalQA
chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type='stuff',
    retriever= retriever,
    input_key = 'query',
    return_source_documents = True,
    chain_type_kwargs={"prompt":PROMPT},
    verbose=True

)

In [ ]:
from IPython.display import display
from IPython.display import Markdown
import textwrap

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
query = "what is indemnity?"
response = chain(query)
to_markdown(response['result'])

In [ ]:
query = "can minors have a life insurance policy?"
response = chain(query)
to_markdown(response['result'])

In [ ]:
query = "what kind of insurance policy is recommended for senior citizens?"
response = chain(query)
print(response)
to_markdown(response['result'])


In [ ]:
query = "under what circumstances can a person withdraw money from their MPF account?"
response = chain(query)
to_markdown(response['result'])

In [ ]:
query = "can someone withdraw money from MPF if they are permanently leaving Hong Kong?"
response = chain(query)
to_markdown(response['result'])